In [121]:
import fasttext
model = fasttext.load_model('./util/cc.en.300.bin')

In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import re

from tqdm import tqdm

In [141]:
import util.normalization as norm
import util.model as models
import util.utility as util

In [142]:
# Opening JSON file
with open("./mcd_result/map_second_mcd.json") as json_file:
    map_second_keys = json.load(json_file)

with open("./mcd_result/map_category_mcd.json") as json_file:
    map_category = json.load(json_file)

with open("./mcd_result/list_first_cycle_mcd.json") as json_file:
    list_first_cycle = json.load(json_file)

In [143]:
df_base = pd.read_csv("./mcd_result/base_mcd_v2.csv")

print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviewer_id       100 non-null    int64 
 1   review            100 non-null    object
 2   summarize         100 non-null    object
 3   aspect            100 non-null    object
 4   token_sentence    100 non-null    object
 5   token_lemma       100 non-null    object
 6   rating            100 non-null    int64 
 7   residu            95 non-null     object
 8   key_score_aspect  98 non-null     object
dtypes: int64(2), object(7)
memory usage: 7.2+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"clear substance, someone spit, everyone, norma...","Why does it look like someone spit on my food,...","why do it look like someone spit on my food, i...",1,"normal transaction, clear substance, everyone,...","situation, normal transaction, positive; drink..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"mcdonalds, atmosphere, staff, food",The staff at McDonalds are always friendly and...,the staff at mcdonalds be always friendly and ...,4,atmosphere,"place, mcdonalds, positive; place, atmosphere,..."
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"line, mobile order, money, refund","Never got the refund in the app., Made a mobil...","never get the refund in the app ., make a mobi...",1,"refund, mobile order, money","situation, line, negative; service, mobile ord..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"service, crispy chicken, customer service, chi...",Crispy chicken sandwich was customer service ...,crispy chicken sandwich be customer service be...,5,crispy chicken,"service, service/customer service, positive; ;..."
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","order, large fry, large meal, close attention,...","I repeat my order 3 times in the drive thru, i...","i repeat my order 3 time in the drive thru, it...",1,"double filet, english, close attention","place, close attention, negative; food, order,..."


In [144]:
def split_all(x):
    temp = []
    for e in x:
        if len(e) == 0:
            continue
        temp.append(tuple(e.split(", ")))
        
    return temp



df_base = df_base.fillna("")

df_base['aspect'] = df_base['aspect'].apply(lambda x: x.split(", "))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: x.split(", "))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: x.split(", "))
df_base['residu'] = df_base['residu'].apply(lambda x: x.split(", "))

df_base['key_score_aspect'] = df_base['key_score_aspect'].apply(lambda x: x.split("; ")).apply(lambda x: split_all(x))

print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviewer_id       100 non-null    int64 
 1   review            100 non-null    object
 2   summarize         100 non-null    object
 3   aspect            100 non-null    object
 4   token_sentence    100 non-null    object
 5   token_lemma       100 non-null    object
 6   rating            100 non-null    int64 
 7   residu            100 non-null    object
 8   key_score_aspect  100 non-null    object
dtypes: int64(2), object(7)
memory usage: 7.2+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[clear substance, someone spit, everyone, norm...",[Why does it look like someone spit on my food...,"[why do it look like someone spit on my food, ...",1,"[normal transaction, clear substance, everyone...","[(situation, normal transaction, positive), (d..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[mcdonalds, atmosphere, staff, food]",[The staff at McDonalds are always friendly an...,[the staff at mcdonalds be always friendly and...,4,[atmosphere],"[(place, mcdonalds, positive), (place, atmosph..."
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[line, mobile order, money, refund]","[Never got the refund in the app., Made a mobi...","[never get the refund in the app ., make a mob...",1,"[refund, mobile order, money]","[(situation, line, negative), (service, mobile..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[service, crispy chicken, customer service, ch...",[Crispy chicken sandwich was customer service...,[crispy chicken sandwich be customer service b...,5,[crispy chicken],"[(service, service/customer service, positive)..."
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, large fry, large meal, close attention...","[I repeat my order 3 times in the drive thru, ...","[i repeat my order 3 time in the drive thru, i...",1,"[double filet, english, close attention]","[(place, close attention, negative), (food, or..."


In [145]:
columns = ['reviewer_id', 'review', 'summarize', 'token_sentence', 'key_score_aspect', 'rating']

df_main = df_base[columns].copy()

df_main['token_sentence'] = df_main['token_sentence'].apply(lambda x: "\n".join("● " + item for item in x))

df_main.head()

,reviewer_id,review,summarize,token_sentence,key_score_aspect,rating
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,● Why does it look like someone spit on my foo...,"[(situation, normal transaction, positive), (d...",1
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,● The staff at McDonalds are always friendly a...,"[(place, mcdonalds, positive), (place, atmosph...",4
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,● Never got the refund in the app.\n● Made a m...,"[(situation, line, negative), (service, mobile...",1
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,● Crispy chicken sandwich was customer servic...,"[(service, service/customer service, positive)...",5
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...",● I repeat my order 3 times in the drive thru\...,"[(place, close attention, negative), (food, or...",1


In [146]:
def custom_text(x):
    temp = {}
    text = ""
    if not x:
        return text
    
    for s in x:
        category, key, sentiment = s

        if category not in temp:
            temp[category] = f"\t- {key} ({sentiment})"
        else:
            temp[category] = temp[category] + "\n" + f"\t- {key} ({sentiment})"



    categories = list(temp.keys())
    for category in categories:
        temp_text = f"● {category}:\n" + temp[category]

        text = text + temp_text + '\n'

    return text

In [147]:
df_main['key_score_aspect'] = df_main['key_score_aspect'].apply(lambda x: custom_text(x))

df_main = df_main.drop('key_score_aspect', axis=1)

In [148]:
df_main.to_csv('./mcd_result/main_mcd.csv', index=False)